In [1]:
import time
import os 

start_time = time.time()

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')


# import yfinance as yf
# import pandas as pd
# import numpy as np

from utils.finance_utils import *
# from utils.finance_utils import get_sector, get_top_n_by_sector, get_corr_pairs_of_stocks

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py

---------------------------------
finance_utils.py successfully loaded, updated last Jan. 27 2025
---------------------------------




In [2]:
filtered_valuation_df = pd.read_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/filtered_valuation_df.csv')
filtered_valuation_df.head()

,marketCap,trailingPE,profitMargins,trailingEps,Tickers,Sharpe_ratios
0,2.885004e+10,64.561430,0.65343,19.45,TPL,2.933049
1,2.900106e+12,46.622047,0.55041,2.54,NVDA,2.117434
2,9.061301e+10,9.031250,0.50511,5.92,MO,1.959634
3,2.877033e+10,16.447030,0.41335,12.46,GDDY,3.093362
4,1.858313e+11,8.540983,0.40711,6.10,HSBC,1.982778


Sector Valuation: Group by sectors, and find best stock in each sector

In [3]:

# Add sector column to the DataFrame
filtered_valuation_df['sector'] = filtered_valuation_df['Tickers'].apply(get_sector)
filtered_valuation_df

,marketCap,trailingPE,profitMargins,trailingEps,Tickers,Sharpe_ratios,sector
0,2.885004e+10,64.561430,0.65343,19.45,TPL,2.933049,Energy
1,2.900106e+12,46.622047,0.55041,2.54,NVDA,2.117434,Technology
2,9.061301e+10,9.031250,0.50511,5.92,MO,1.959634,Consumer Defensive
3,2.877033e+10,16.447030,0.41335,12.46,GDDY,3.093362,Technology
4,1.858313e+11,8.540983,0.40711,6.10,HSBC,1.982778,Financial Services
5,1.263539e+11,48.221157,0.40294,2.08,ANET,2.064880,Technology
6,5.175554e+10,11.981132,0.39410,4.24,MPLX,3.012816,Energy
7,2.719178e+10,9.070129,0.35139,7.70,SYF,3.124226,Financial Services
8,4.294568e+10,8.401575,0.34031,1.27,NWG,2.751161,Financial Services
9,7.438818e+11,13.456477,0.33163,19.76,JPM,2.262297,Financial Services


In [4]:
# possibly change group by  
print('\n----Filtering Df----')


top_stocks_on_profit = get_top_n_by_sector(filtered_valuation_df, filter_var='profitMargins', top_n=3)
top_stocks_on_sharpe = get_top_n_by_sector(filtered_valuation_df, filter_var='Sharpe_ratios', top_n=3)

print(f'Filtered based on: Profit Margins')
print(f"Sector Count: {len(top_stocks_on_profit['sector'].unique())}")
print(f"We have the following sector(s): \n{top_stocks_on_profit['sector'].unique()}")

print(f'\nFiltered based on: Sharpe Ratio')
print(f"Sector Count: {len(top_stocks_on_sharpe['sector'].unique())}")
print(f"We have the following sector(s): \n{top_stocks_on_sharpe['sector'].unique()}")


----Filtering Df----
Filtered based on: Profit Margins
Sector Count: 7
We have the following sector(s): 
['Communication Services' 'Consumer Cyclical' 'Consumer Defensive'
 'Energy' 'Financial Services' 'Healthcare' 'Technology']

Filtered based on: Sharpe Ratio
Sector Count: 7
We have the following sector(s): 
['Communication Services' 'Consumer Cyclical' 'Consumer Defensive'
 'Energy' 'Financial Services' 'Healthcare' 'Technology']


Diversifaction: Find negative correlated portfolios
    Gotta get histoical price, and do it on that

In [5]:

profit_tickers_to_do_corr = top_stocks_on_profit['Tickers'].tolist()
top_profit_corr_pairs = get_corr_pairs_of_stocks(profit_tickers_to_do_corr)

sharpe_tickers_to_do_corr = top_stocks_on_sharpe['Tickers'].tolist()
top_sharpe_corr_pairs = get_corr_pairs_of_stocks(sharpe_tickers_to_do_corr)


[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  14 of 14 completed


Get Correlation, but need to find a way to find optimal pair, such that corr is low, all around portofolio

In [6]:
corr_profit_pairs = top_profit_corr_pairs.index.to_list()
corr_sharpe_pairs = top_sharpe_corr_pairs.index.to_list()

In [ ]:
def separate_corr_pairs(corr_pairs: list, top_n: int) -> np.array:
    """
    Extracts and returns unique stock pairs from a list of correlation pairs.

    Args:
        corr_pairs (list): List of stock correlation pairs in 'StockA_StockB' format.
        top_n (int): Number of unique stock symbols to return. Defaults to 8.

    Returns:
        np.array: Array of unique stock symbols from the correlation pairs.
    """
    pair_list = [pair.split('_') for pair in corr_pairs]  # Split stock pairs
    pair_list_1d = np.array(pair_list).flatten()  # Flatten into a 1D array
    unique_values = pd.unique(pair_list_1d)[:top_n]  # Get top unique stock symbols
    
    return unique_values

top_n_for_stocks = 8
mpt_stocks_for_sharpe = separate_corr_pairs(corr_sharpe_pairs, top_n=top_n_for_stocks)
mpt_stocks_for_profit = separate_corr_pairs(corr_profit_pairs, top_n=top_n_for_stocks)

In [8]:
print(f'Top Stocks based on Sharpe: {mpt_stocks_for_sharpe}')
print(f'Top Stocks based on Profit Margin: {mpt_stocks_for_profit}')

Top Stocks based on Sharpe: ['PM' 'TCOM' 'MO' 'APP' 'NVDA' 'HOOD' 'ISRG' 'NWG']
Top Stocks based on Profit Margin: ['CQP' 'NVDA' 'PM' 'NWG' 'MO' 'TCOM' 'HSBC' 'ANET']


Do mpt on selected stocks above